### Paper Name: Feature requests-based recommendation of software refactorings
#### Authors: Ally S. Nyamawe · Hui Liu · Nan Niu · Qasim Umer · Zhendong Niu

## 
#### Create Dataset

In [1]:
import pandas as pd
import pickle

if __name__ == '__main__':

    f = open('/Users/kritya/DSCI_Project/project-dsci-644-group-7-dsci_7/src/dataset/final_dataset.pickle', 'rb')
    dataset = pickle.load(f)
    f.close()
    print("dataset laoded.....")
    print('==========>')


    binary_class_dataset = []
    for index in range(len(dataset)):
        task = []
        task.append(dataset[index][0])
        if str(dataset[index][1][0]).strip() == 'none':
            ref = 0
        else:
            ref = 1
        task.append(ref)

        binary_class_dataset.append(task)

    f = open('/Users/kritya/DSCI_Project/project-dsci-644-group-7-dsci_7/src/dataset/binary_class_dataset.pickle', 'wb')
    pickle.dump(binary_class_dataset,f)
    f.close()
    print("dataset saved.....")
    print('==========>')
    
    
    print(binary_class_dataset[0])


/Users/kritya/.pyenv/versions/3.7.5/lib/python3.7/site-packages/pandas/compat/__init__.py:124: UserWarning: Could not import the lzma module. Your installed Python is incomplete. Attempting to use lzma compression will result in a RuntimeError.
  warnings.warn(msg)


dataset laoded.....
==========>
dataset saved.....
==========>
[' anoth blueprint sampl contribut day test blueprint mbean gener complex blueprint sampl sampl user learn defin nest compon blueprint xml run sampl besid osgi framework bundl requir coreopt mavenbundl groupid org apach felix artifactid org apach felix configadmin versionasinproject coreopt mavenbundl groupid org apach felix artifactid org apach felix eventadmin versionasinproject coreopt mavenbundl groupid org ops4j pax log artifactid pax log api versionasinproject coreopt mavenbundl groupid org ops4j pax log artifactid pax log servic versionasinproject coreopt mavenbundl groupid org apach ari blueprint artifactid org apach ari blueprint versionasinproject coreopt mavenbundl groupid org apach ari artifactid org apach ari util coreopt mavenbundl groupid org apach ari jmx artifactid ari jmx blueprint versionasinproject', 0]


##
#### Feature words

In [2]:
import pickle
from nltk import word_tokenize, pos_tag
from nltk.stem import WordNetLemmatizer
import re
from collections import Counter
import string
import numpy as np
import math

def clean_text(text):
    text = re.sub(r'\w+:\/{2}[\d\w-]+(\.[\d\w-]+)*(?:(?:\/[^\s/]*))*', '', text)
    text.lower();
    # Get the difference of all ASCII characters from the set of printable characters
    nonprintable = set([chr(i) for i in range(128)]).difference(string.printable)
    # Use translate to remove all non-printable characters
    return text.translate({ord(character): None for character in nonprintable})

def tokenization(text):
    return word_tokenize(text)

def pos_tagging(text):

    return pos_tag(text)

def lemmatization(pos_tags):
    adjective_tags = ['JJ', 'JJR', 'JJS']
    wordnet_lemmatizer = WordNetLemmatizer()
    lemmatized_text = []
    for word in pos_tags:
        if word[1] in adjective_tags:
            lemmatized_text.append(str(wordnet_lemmatizer.lemmatize(word[0], pos="a")))
        else:
            lemmatized_text.append(str(wordnet_lemmatizer.lemmatize(word[0])))  # default POS = noun
    return lemmatized_text

def stop_word_removal(pos_tags_lem, lem_text):
    stopwords = []

    wanted_POS = ['NN', 'NNS', 'NNP', 'NNPS', 'JJ', 'JJR', 'JJS', 'VBG', 'FW']

    for word in pos_tags_lem:
        if word[1] not in wanted_POS:
            stopwords.append(word[0])

    punctuations = list(str(string.punctuation))

    stopwords = stopwords + punctuations

    stopword_file = open("./dataset/long_stopwords.txt", "r")
    # Source = https://www.ranks.nl/stopwords

    lots_of_stopwords = []

    for line in stopword_file.readlines():
        lots_of_stopwords.append(str(line.strip()))

    stopwords_plus = []
    stopwords_plus = stopwords + lots_of_stopwords
    stopwords_plus = set(stopwords_plus)

    processed_text = []
    for word in lem_text:
        if word not in stopwords_plus:
            processed_text.append(word)

    return processed_text, stopwords_plus

if __name__ == '__main__':
    load = open("/Users/kritya/DSCI_Project/project-dsci-644-group-7-dsci_7/src/dataset/binary_class_dataset.pickle", "rb")
    tasks = pickle.load(load)
    load.close()
    print(tasks[0])
    print("tasks loaded.....")

    preprocessed_tasks = []

    all_words = []
    freq_words_without_freq = []

    for index in range(len(tasks)):

        # without preprocessing
        old_task = tasks[index]
        text = tasks[index][0].lower()
        cleaned_text = clean_text(text)
        words = re.findall(r'\w+', cleaned_text)

               # without preprocessing
        for item in words:

            all_words.append(item)


    counts = Counter(all_words).most_common(5000)

    for index in range(len(counts)):
        freq_words_without_freq.append(counts[index][0])


    file = open("/Users/kritya/DSCI_Project/project-dsci-644-group-7-dsci_7/src/dataset/freq_words_without_preprocessing.pickle", "wb")
    pickle.dump(freq_words_without_freq, file)
    file.close()
    print("all_phrases_without_freq saved......")




[' anoth blueprint sampl contribut day test blueprint mbean gener complex blueprint sampl sampl user learn defin nest compon blueprint xml run sampl besid osgi framework bundl requir coreopt mavenbundl groupid org apach felix artifactid org apach felix configadmin versionasinproject coreopt mavenbundl groupid org apach felix artifactid org apach felix eventadmin versionasinproject coreopt mavenbundl groupid org ops4j pax log artifactid pax log api versionasinproject coreopt mavenbundl groupid org ops4j pax log artifactid pax log servic versionasinproject coreopt mavenbundl groupid org apach ari blueprint artifactid org apach ari blueprint versionasinproject coreopt mavenbundl groupid org apach ari artifactid org apach ari util coreopt mavenbundl groupid org apach ari jmx artifactid ari jmx blueprint versionasinproject', 0]
tasks loaded.....
all_phrases_without_freq saved......


## 
#### Feature sets generation

In [3]:
import pickle
import random

def load_dataset():
    file = open("/Users/kritya/DSCI_Project/project-dsci-644-group-7-dsci_7/src/dataset/shuffle_dataset.pickle", "rb")
    dataset = pickle.load(file)
    file.close()
    print("dataset of length " , len(dataset) , " is loaded.....")
    return dataset

def load_features():
    # file = open("./dataset/features.pickle", "rb")
    file = open("/Users/kritya/DSCI_Project/project-dsci-644-group-7-dsci_7/src/dataset/features_without_pp.pickle", "rb")
    features = pickle.load(file)
    file.close()
    print("features of length " , len(features) , " is loaded.....")
    # print(features[:100])
    return features

def find_features(w_features, t_words):
    words = t_words
    features = {}
    for f in w_features:
        features[f] = 0

    for f in features:
        if f in words:
            features[f] += 1
    return features


def make_featuresets(word_features):

    documents_f = open("/Users/kritya/DSCI_Project/project-dsci-644-group-7-dsci_7/src/T&T/testing.pickle", "rb")
    testing_files = pickle.load(documents_f)
    documents_f.close()
    print(str(len(testing_files)) + " testing files loaded.....")

    testing_featureset = []

    for index in range(len(testing_files)):
        testing_featureset.append(
            [find_features(word_features, testing_files[index][2]),
             testing_files[index][1]])

    save_featuresets = open("/Users/kritya/DSCI_Project/project-dsci-644-group-7-dsci_7/src/feature_modeling/testing_featureset.pickle", "wb")
    pickle.dump(testing_featureset, save_featuresets)
    save_featuresets.close()
    print("testing featuresets saved.....")
    print("===============>")

    for t_index in range(10):
        documents_f = open("/Users/kritya/DSCI_Project/project-dsci-644-group-7-dsci_7/src/T&T/training" + str(t_index + 1) +".pickle", "rb")
        training_files = pickle.load(documents_f)
        documents_f.close()
        print(str(len(training_files)) + " training files loaded.....")

        training_featureset = []

        for index in range(len(training_files)):
            training_featureset.append(
                [find_features(word_features, training_files[index][2]),
                 training_files[index][1]])

        save_featuresets = open("/Users/kritya/DSCI_Project/project-dsci-644-group-7-dsci_7/src/feature_modeling/training_featureset" + str(t_index + 1) + ".pickle","wb")
        pickle.dump(training_featureset, save_featuresets)
        save_featuresets.close()
        print("training featuresets " + str(t_index + 1) + " saved.....")
        print("===============>")


if __name__ == '__main__':
    # dataset = load_dataset()
    features = load_features()
    # saperate_training_and_testing_data(dataset)
    make_featuresets(features)

features of length  5000  is loaded.....
1336 testing files loaded.....
testing featuresets saved.....
===============>
1200 training files loaded.....
training featuresets 1 saved.....
===============>
1200 training files loaded.....
training featuresets 2 saved.....
===============>
1200 training files loaded.....
training featuresets 3 saved.....
===============>
1200 training files loaded.....
training featuresets 4 saved.....
===============>
1200 training files loaded.....
training featuresets 5 saved.....
===============>
1200 training files loaded.....
training featuresets 6 saved.....
===============>
1200 training files loaded.....
training featuresets 7 saved.....
===============>
1200 training files loaded.....
training featuresets 8 saved.....
===============>
1200 training files loaded.....
training featuresets 9 saved.....
===============>
1228 training files loaded.....
training featuresets 10 saved.....
===============>


## 
#### Refactoring Prediction

In [4]:
import nltk
from nltk.classify.scikitlearn import SklearnClassifier
import pickle
from sklearn.naive_bayes import MultinomialNB, BernoulliNB
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import LinearSVC, SVC
from sklearn.preprocessing import LabelEncoder

classifiers = ['SVM', 'NB', 'MNB', 'BNB', 'LR', 'RF']
# classifiers = ['SVM']
# data loading
featuresets_f = open("/Users/kritya/DSCI_Project/project-dsci-644-group-7-dsci_7/src/feature_modeling/testing_featureset.pickle", "rb")
testing_set = pickle.load(featuresets_f)
featuresets_f.close()
print("lenght of testing dataset: ", len(testing_set))

for t_index in range(10):

    print("iteration " + str(t_index + 1))
    featuresets_f = open("/Users/kritya/DSCI_Project/project-dsci-644-group-7-dsci_7/src/feature_modeling/training_featureset" + str(t_index + 1) + ".pickle", "rb")
    training = pickle.load(featuresets_f)
    featuresets_f.close()
    print("lenght of training dataset: ", len(training))

    for cls in classifiers:
        if cls == 'SVM':
            classifier = SklearnClassifier(LinearSVC())
            classifier.train(training)
        elif cls == 'NB':
            classifier = nltk.NaiveBayesClassifier.train(training)
            classifier.train(training)
        elif cls == 'MNB':
            classifier = SklearnClassifier(MultinomialNB())
            classifier.train(training)
        elif cls == 'BNB':
            classifier = SklearnClassifier(BernoulliNB())
            classifier.train(training)
        elif cls == 'LR':
            classifier = SklearnClassifier(LogisticRegression())
            classifier.train(training)
        elif cls == 'RF':
            classifier = SklearnClassifier(RandomForestClassifier())
            classifier.train(training)
        # prediction
        y_true, y_pred = [], []

        for i, (feats, label_true) in enumerate(testing_set):
            label_pred = classifier.classify(feats)
            y_true.append(label_true)
            y_pred.append(label_pred)

        # save_classifier = open("./trained_classifiers/LRall.pickle", "wb")
        save_classifier = open("/Users/kritya/DSCI_Project/project-dsci-644-group-7-dsci_7/src/trained_classifiers" + cls + str(t_index + 1) + ".pickle", "wb")
        pickle.dump(classifier, save_classifier)
        save_classifier.close()

        # save_classifier = open("./y_true_pred/y_true_LRall.pickle", "wb")
        save_classifier = open("/Users/kritya/DSCI_Project/project-dsci-644-group-7-dsci_7/src/y_true_pred/y_true" + cls + str(t_index + 1) +  ".pickle", "wb")
        pickle.dump(y_true, save_classifier)
        save_classifier.close()

        # save_classifier = open("./y_true_pred/y_pred_LRall.pickle", "wb")
        save_classifier = open("/Users/kritya/DSCI_Project/project-dsci-644-group-7-dsci_7/src/y_true_pred/y_pred" + cls + str(t_index + 1) +  ".pickle", "wb")
        pickle.dump(y_pred, save_classifier)
        save_classifier.close()

        print(cls + " for iteration " + str(t_index + 1) + " done.....")
        print("=========>>")

lenght of testing dataset:  1336
iteration 1
lenght of training dataset:  1200
SVM for iteration 1 done.....
=========>>
NB for iteration 1 done.....
=========>>
MNB for iteration 1 done.....
=========>>
BNB for iteration 1 done.....
=========>>
LR for iteration 1 done.....
=========>>
RF for iteration 1 done.....
=========>>
iteration 2
lenght of training dataset:  1200
SVM for iteration 2 done.....
=========>>
NB for iteration 2 done.....
=========>>
MNB for iteration 2 done.....
=========>>
BNB for iteration 2 done.....
=========>>
LR for iteration 2 done.....
=========>>
RF for iteration 2 done.....
=========>>
iteration 3
lenght of training dataset:  1200


/Users/kritya/.pyenv/versions/3.7.5/lib/python3.7/site-packages/sklearn/svm/_base.py:1208: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  ConvergenceWarning,


SVM for iteration 3 done.....
=========>>
NB for iteration 3 done.....
=========>>
MNB for iteration 3 done.....
=========>>
BNB for iteration 3 done.....
=========>>
LR for iteration 3 done.....
=========>>
RF for iteration 3 done.....
=========>>
iteration 4
lenght of training dataset:  1200
SVM for iteration 4 done.....
=========>>
NB for iteration 4 done.....
=========>>
MNB for iteration 4 done.....
=========>>
BNB for iteration 4 done.....
=========>>
LR for iteration 4 done.....
=========>>
RF for iteration 4 done.....
=========>>
iteration 5
lenght of training dataset:  1200
SVM for iteration 5 done.....
=========>>
NB for iteration 5 done.....
=========>>
MNB for iteration 5 done.....
=========>>
BNB for iteration 5 done.....
=========>>
LR for iteration 5 done.....
=========>>
RF for iteration 5 done.....
=========>>
iteration 6
lenght of training dataset:  1200
SVM for iteration 6 done.....
=========>>
NB for iteration 6 done.....
=========>>
MNB for iteration 6 done.....
==

## 
#### Multi-label Classifier

In [7]:
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.model_selection import train_test_split
from imblearn.over_sampling import RandomOverSampler
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import LinearSVC
from sklearn.linear_model import LogisticRegression, SGDClassifier
from sklearn.multiclass import OneVsRestClassifier
from sklearn.metrics import hamming_loss, accuracy_score, precision_score, recall_score, f1_score, classification_report
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import pyodbc
from nltk.stem import PorterStemmer
from nltk.corpus import stopwords
import re


df_text = pd.read_csv('/Users/kritya/DSCI_Project/project-dsci-644-group-7-dsci_7/src/dataset/TextPreprocessed.csv', encoding='iso-8859-1')
# print(df_text.head())
df_tags = pd.read_csv('/Users/kritya/DSCI_Project/project-dsci-644-group-7-dsci_7/src/dataset/Tag.csv', encoding='iso-8859-1')

num_classes = 14
grouped_tags = df_tags.groupby("Tag").size().reset_index(name='count')
most_common_tags = grouped_tags.nlargest(num_classes, columns="count")
df_tags.Tag = df_tags.Tag.apply(lambda tag : tag if tag in most_common_tags.Tag.values else None)
df_tags = df_tags.dropna()

counts = df_tags.Tag.value_counts()
firstlast = counts[:5].append(counts[-5:])
firstlast.reset_index(name="count")

# print(firstlast)

def tags_for_question(question_id):
    return df_tags[df_tags['Id'] == question_id].Tag.values

def add_tags_column(row):
    row['Tags'] = tags_for_question(row['Id'])
    return row

df_questions = df_text.apply(add_tags_column, axis=1)

multilabel_binarizer = MultiLabelBinarizer()
multilabel_binarizer.fit(df_questions.Tags)
Y = multilabel_binarizer.transform(df_questions.Tags)

count_vect = CountVectorizer()
X_counts = count_vect.fit_transform(df_questions.Text.values.astype('U'))

tfidf_transformer = TfidfTransformer()
X_tfidf = tfidf_transformer.fit_transform(X_counts)

ros = RandomOverSampler(random_state=9000)
X_tfidf_resampled, Y_tfidf_resampled = ros.fit_resample(X_tfidf, Y)

x_train_tfidf, x_test_tfidf, y_train_tfidf, y_test_tfidf = train_test_split(X_tfidf_resampled, Y_tfidf_resampled, test_size=0.2, random_state=9000)


def hamming_score(y_true, y_pred, normalize=True, sample_weight=None):
    print(y_true.shape[0])
    print(y_pred)

    acc_list = []
    for i in range(y_true.shape[0]):
        set_true = set(np.where(y_true[i])[0])
        set_pred = set(np.where(y_pred[i])[0])
        tmp_a = None
        if len(set_true) == 0 and len(set_pred) == 0:
            tmp_a = 1
        else:
            # tmp_a = len(set_true.union(set_pred))
            tmp_a = len(set_true.intersection(set_pred))/float(len(set_true.union(set_pred)) )
        acc_list.append(tmp_a)
    # print(acc_list)
    return np.mean(acc_list)

def print_score(y_pred, clf):
    print("Clf: ", clf.__class__.__name__)
    # print("Hamming loss: {}".format(hamming_loss(y_test_tfidf, y_pred)))
    print("Hamming score: {}".format(hamming_score(y_test_tfidf, y_pred)))
    # print('Subset accuracy: {0}'.format(accuracy_score(y_test_tfidf, y_pred, normalize=True, sample_weight=None)))
    # print('Subset precision: {0}'.format(precision_score(y_test_tfidf, y_pred, average='samples')))
    print("---")

sgd = SGDClassifier(loss='hinge', penalty='l2', alpha=1e-3, random_state=42, max_iter=6, tol=None)
lr = LogisticRegression()
mn = MultinomialNB()
svm = LinearSVC()
classifer_models = [sgd, lr, mn, svm]
for classifier in classifer_models:
    clf = OneVsRestClassifier(classifier)
    clf.fit(x_train_tfidf, y_train_tfidf)
    y_pred = clf.predict(x_test_tfidf)
    # print_score(y_pred, classifier)
    print("Classification Report of ", classifier)
    print(classification_report(y_test_tfidf, y_pred))

Classification Report of  SGDClassifier(alpha=0.001, max_iter=6, random_state=42, tol=None)
              precision    recall  f1-score   support

           0       0.00      0.00      0.00       535
           1       0.00      0.00      0.00       525
           2       0.00      0.00      0.00       531
           3       0.00      0.00      0.00       539
           4       0.96      0.04      0.08       511
           5       0.00      0.00      0.00       519
           6       0.00      0.00      0.00       544
           7       0.00      0.00      0.00       499
           8       0.00      0.00      0.00       494
           9       0.78      0.03      0.05       512
          10       1.00      0.01      0.02       519
          11       0.00      0.00      0.00       507
          12       0.00      0.00      0.00       547
          13       0.00      0.00      0.00       518

   micro avg       0.89      0.01      0.01      7300
   macro avg       0.20      0.01      0.0

/Users/kritya/.pyenv/versions/3.7.5/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/kritya/.pyenv/versions/3.7.5/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Classification Report of  LogisticRegression()
              precision    recall  f1-score   support

           0       0.94      0.87      0.91       535
           1       0.00      0.00      0.00       525
           2       0.95      0.83      0.89       531
           3       0.94      0.06      0.12       539
           4       0.94      0.59      0.73       511
           5       0.90      0.05      0.10       519
           6       1.00      0.09      0.17       544
           7       1.00      0.01      0.02       499
           8       0.94      0.81      0.87       494
           9       0.95      0.48      0.64       512
          10       0.97      0.78      0.87       519
          11       0.95      0.81      0.88       507
          12       1.00      0.08      0.15       547
          13       1.00      0.00      0.01       518

   micro avg       0.95      0.39      0.55      7300
   macro avg       0.89      0.39      0.45      7300
weighted avg       0.89      0.39

/Users/kritya/.pyenv/versions/3.7.5/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/kritya/.pyenv/versions/3.7.5/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Classification Report of  MultinomialNB()
              precision    recall  f1-score   support

           0       0.91      0.31      0.46       535
           1       0.27      0.01      0.01       525
           2       0.93      0.34      0.50       531
           3       0.64      0.03      0.06       539
           4       0.85      0.17      0.29       511
           5       0.73      0.05      0.10       519
           6       0.76      0.07      0.12       544
           7       0.64      0.04      0.07       499
           8       0.91      0.21      0.35       494
           9       0.83      0.09      0.16       512
          10       0.95      0.38      0.54       519
          11       0.94      0.25      0.39       507
          12       0.69      0.05      0.10       547
          13       0.36      0.01      0.02       518

   micro avg       0.87      0.14      0.25      7300
   macro avg       0.74      0.14      0.23      7300
weighted avg       0.74      0.14     

/Users/kritya/.pyenv/versions/3.7.5/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Classification Report of  LinearSVC()
              precision    recall  f1-score   support

           0       0.91      0.94      0.92       535
           1       0.57      0.07      0.13       525
           2       0.90      0.92      0.91       531
           3       0.90      0.66      0.76       539
           4       0.89      0.87      0.88       511
           5       0.83      0.65      0.73       519
           6       0.90      0.71      0.79       544
           7       0.84      0.45      0.58       499
           8       0.90      0.92      0.91       494
           9       0.92      0.77      0.84       512
          10       0.93      0.82      0.87       519
          11       0.94      0.82      0.88       507
          12       0.90      0.65      0.75       547
          13       0.71      0.21      0.33       518

   micro avg       0.89      0.68      0.77      7300
   macro avg       0.86      0.68      0.73      7300
weighted avg       0.86      0.68      0.7

/Users/kritya/.pyenv/versions/3.7.5/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
